In [1]:
import keras
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.models import Sequential
from keras.utils import np_utils
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import r2_score
import numpy as np

Using TensorFlow backend.
F:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
F:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
F:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
F:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type

In [3]:
from keras.models import Model
from keras.losses import mse
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate,Conv2D,MaxPooling2D,BatchNormalization,UpSampling2D,Lambda,Reshape
from keras import backend as K

In [4]:
def imshow(image, cm='hot') :
    sns.set()
    sns.heatmap(image, cmap=cm)

In [5]:
tf_x = K.placeholder(dtype=tf.float32,shape=(None, 200, 200, 4))
tf_y = K.placeholder(dtype=tf.float32,shape=(None, 200, 200, 1))
tf_z = K.placeholder(dtype=tf.float32,shape=(None, 7))
is_t = K.placeholder(dtype=tf.bool,)

In [6]:
# 设定随机数
seed = 7
np.random.seed(seed)
# 类别个数
num_classes = None
# 图片大小
lng = 200
# 属性维度
att_d = 4
int_shp = (lng, lng, att_d)
cut_num = 300

In [7]:
pred_x_name = 'E:/20190924_match_pro/MHNN/data/scale_data/pred_x2.npy'
pred_y_name = 'E:/20190924_match_pro/MHNN/data/scale_data/pred_y2.npy'
pred_z_name = 'E:/20190924_match_pro/MHNN/data/scale_data/pred_z2.npy'
train_x_name= 'E:/20190924_match_pro/MHNN/data/scale_data/train_x2.npy'
train_y_name= 'E:/20190924_match_pro/MHNN/data/scale_data/train_y2.npy'
train_z_name= 'E:/20190924_match_pro/MHNN/data/scale_data/train_z2.npy'

print('loading train_x')
train_x = np.load(train_x_name)
print('loading train_y')
train_y = np.load(train_y_name)
print('loading train_z')
train_z = np.load(train_z_name)
print('loading pred_x')
pred_x  = np.load(pred_x_name)
print('loading pred_y')
pred_y  = np.load(pred_y_name)
print('loading pred_z')
pred_z  = np.load(pred_z_name)

loading train_x
loading train_y
loading train_z
loading pred_x
loading pred_y
loading pred_z


In [8]:
print(train_x.shape)
print(train_y.shape)
print(train_z.shape)

print(pred_x.shape)

(300, 200, 200, 4)
(300, 200, 200, 1)
(300, 7)
(1300, 200, 200, 4)


In [9]:
##定义模型
def MHNN_model():    
  
    main_input = Input(shape = (200,200,4), name='main_input')
    
    conv1 = Conv2D(filters=16, strides=1, kernel_size=3,activation='relu', padding='same')(main_input)
    conv1 = Conv2D(filters=16, strides=1, kernel_size=3,activation='relu', padding='same')(conv1)
    conv1 = Conv2D(filters=16, strides=1, kernel_size=3,activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    BN1   = BatchNormalization()(pool1)
    
    conv2 = Conv2D(filters=64, strides=1, kernel_size=3,activation='relu', padding='same')(BN1)
    conv2 = Conv2D(filters=64, strides=1, kernel_size=3,activation='relu', padding='same')(conv2)
    conv2 = Conv2D(filters=64, strides=1, kernel_size=3,activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    BN2   = BatchNormalization()(pool2)
    
    conv3 = Conv2D(filters=64, strides=1, kernel_size=3,activation='relu', padding='same')(BN2)
    conv3 = Conv2D(filters=64, strides=1, kernel_size=3,activation='relu', padding='same')(conv3)
    conv3 = Conv2D(filters=64, strides=1, kernel_size=3,activation='relu', padding='same')(conv3)
    BN3   = BatchNormalization()(conv3)
    
    up1   = UpSampling2D(size = (2,2))(BN3)
    up1   = Conv2D(filters=64, strides=1, kernel_size=3,activation='relu', padding='same')(up1)
    
    merge1 = concatenate([conv2,up1], axis = 3)
    
    conv4 = Conv2D(filters=64, strides=1, kernel_size=3,activation='relu', padding='same')(merge1)
    conv4 = Conv2D(filters=64, strides=1, kernel_size=3,activation='relu', padding='same')(conv4)
    conv4 = Conv2D(filters=64, strides=1, kernel_size=3,activation='relu', padding='same')(conv4)
    
    up2   = UpSampling2D(size = (2,2))(conv4)
    up2   = Conv2D(filters=16, strides=1, kernel_size=3,activation='relu', padding='same')(up2)
    
    conv5 = Conv2D(filters=16, strides=1, kernel_size=3,activation='relu', padding='same')(up2)
    conv5 = Conv2D(filters=16, strides=1, kernel_size=3,activation='relu', padding='same')(conv5)
    conv5 = Conv2D(filters=16, strides=1, kernel_size=3,activation='relu', padding='same')(conv5)
    conv5 = Conv2D(filters=4, strides=1, kernel_size=3,activation='relu', padding='same')(conv5)
    
    output1 = Conv2D(filters=1, strides=1, kernel_size=3, activation = 'sigmoid', padding = 'same')(conv5)
    
    output1_ = tf.reshape(output1, [-1, conv5.shape[1]*conv5.shape[2]])
    tf_y_    = tf.reshape(tf_y, [-1, conv5.shape[1]*conv5.shape[2]])

    
#     tf_x       = Conv2D(filters=4, strides=1, kernel_size=3,activation='relu', padding='same')(tf_x_input)
    tf_x   = Input(shape = (200,200,1))
    merge2 = concatenate([tf_x,output1], axis = 3)
#     merge2 = Lambda(lambda x: concatenate([tf_x,output1], axis = 3))
    
    conv6 = Conv2D(filters=16, strides=1, kernel_size=3,activation='relu', padding='same')(merge2)
    conv6 = Conv2D(filters=16, strides=1, kernel_size=3,activation='relu', padding='same')(conv6)
    conv6 = Conv2D(filters=16, strides=1, kernel_size=3,activation='relu', padding='same')(conv6)
    pool6 = MaxPooling2D(pool_size=(2, 2))(conv6)
    BN6   = BatchNormalization()(pool6)
    
    merge3 = concatenate([pool6, pool1, conv4], 3) # [pool5, pool1, conv4]
    
    conv7 = Conv2D(filters=16, strides=1, kernel_size=3,activation='relu', padding='same')(merge3)
    conv7 = Conv2D(filters=16, strides=1, kernel_size=3,activation='relu', padding='same')(conv7)
    conv7 = Conv2D(filters=16, strides=1, kernel_size=3,activation='relu', padding='same')(conv7)
    pool7 = MaxPooling2D(pool_size=(2, 2))(conv7)
    
    merge4 = concatenate([pool7, pool2, conv3], 3)
    
    conv8 = Conv2D(filters=16, strides=1, kernel_size=3,activation='relu', padding='same')(merge4)
    conv8 = Conv2D(filters=16, strides=1, kernel_size=3,activation='relu', padding='same')(conv8)
    conv8 = Conv2D(filters=16, strides=1, kernel_size=3,activation='relu', padding='same')(conv8)
    pool8 = MaxPooling2D(pool_size=(2, 2))(conv8)
    
    
#     dense8 = Lambda(lambda x: tf.reshape(pool8, [-1, 25*25*16]))
#     input3 = Input()

    
#     dense8 = Reshape((-1,25*25*16))(pool8)
    
    dense8 = Dense(2048, activation='relu')(pool8)
    BN8    = BatchNormalization()(dense8)
    
    dense9 = Dense(256, activation='relu')(BN8)
    BN9    = BatchNormalization()(dense9)
    
#     output2 = Dense(7)(BN9)
    output2 = Reshape((200,200,4))(BN9)
    model = Model(inputs=[main_input,tf_x], outputs=[output2])
    
    
    
#     def MHNN_loss_function(y_true, y_pred):
        
#         c = BinaryCrossentropy(tf_y_, output1_)
#         r = mse(output2, tf_z)
#         vc = tf.cast(c,dtype=tf.float32)
#         vr = tf.cast(a,dtype=tf.float32)
        
#         loss_cr = vr * 0.1 + vc * 0.9 
#         return loss_cr
        
        
    
    
    
    model.compile(loss='mse', metrics=['accuracy'], optimizer='rmsprop')
    return model

In [10]:
model = MHNN_model()
model.summary()











__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, 200, 200, 4)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 200, 200, 16) 592         main_input[0][0]                 
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 200, 200, 16) 2320        conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 200, 200, 16) 2320        conv2d_2[0][0]                   
__________________________________________________________________________________________________
ma

In [11]:
# 定义损失函数以及准确率回收
class Loss_History(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = {'batch': [], 'epoch': []}
    def on_batch_end(self, batch, logs={}):
        self.losses['batch'].append(logs.get('loss'))
    def on_epoch_end(self, batch, logs={}):
        self.losses['epoch'].append(logs.get('loss'))

In [12]:
# 调整
history = Loss_History()
model.fit([train_x,train_y], pred_x[0:300,:], epochs=50, batch_size=5, shuffle=False)

# print('saving model')
# model.save('E:/20190924_match_pro/classification/model/20190402_vgg_3c_10e_116.h5')

Epoch 1/50


AbortedError: Operation received an exception:Status: 5, message: could not create a view primitive descriptor, in file tensorflow/core/kernels/mkl_slice_op.cc:433
	 [[{{node training/RMSprop/gradients/concatenate_2/concat_grad/Slice_1}}]]